Esse código é um script em Python que gera consultas SQL baseado em tabelas e relações do TOTVS RM.


In [1]:
# Importações

import json
import os.path

# Possivelmente tem que instalar
# pip install numpy pandas
import numpy as np  # Para operações numéricas e manipulação de arrays
import pandas as pd  # Para manipulação de dados tabulares


As planilhas `GDIC.XLSX` e `GLINKSREL.XLSX` devem ser geradas no seu sistema atual com as seguintes SQL

```sql
SELECT TABELA,
       COLUNA,
       DESCRICAO
FROM   GDIC (NOLOCK) /* Lista tabelas do sistema, seus campos e suas descrições */
```

```sql
SELECT MASTERTABLE,
       CHILDTABLE,
       MASTERFIELD,
       CHILDFIELD
FROM   GLINKSREL (NOLOCK) /* Lista relacionamentos entre tabelas */
```


In [2]:
# Leitura de Tabelas e Relacionamentos


def le_arquivo_excel(caminho):
    # Procura pelo arquvio excel
    if not os.path.isfile(caminho):
        raise Exception("Arquivo não existe.")

    caminho_pickle = os.path.splitext(caminho)[0] + ".pkl"

    # Procura pelo arquivo pickle (excel ja processado pelo pandas)
    if os.path.isfile(caminho_pickle):
        return pd.read_pickle(caminho_pickle)
    else:
        # Se não encontrou
        # Processa o excel com o pandas
        df = pd.read_excel(io=caminho).dropna().astype(str)

        # Converte os valores das seguintes colunas em maiusculas
        for coluna in [
            "TABELA",
            "COLUNA",
            "MASTERTABLE",
            "CHILDTABLE",
            "MASTERFIELD",
            "CHILDFIELD",
        ]:
            try:
                df[coluna] = df[coluna].str.upper()
            except KeyError:
                pass

        # Substitui ';' por ',' e apaga caracteres invalidos nas seguintes colunas
        for coluna in ["MASTERFIELD", "CHILDFIELD"]:
            try:
                df[coluna] = df[coluna].str.replace(";", ",")
                df[coluna] = df[coluna].str.replace(
                    r"[^0-9A-Z,_]", "", regex=True
                )
            except KeyError:
                pass

        # Salva como arquivo pickle
        df.to_pickle(caminho_pickle)
        return df


versao_rm = "2402_105"

tabelas = le_arquivo_excel(
    os.path.join(os.getcwd(), "dados", f"GDIC_TOTVS_RM_{versao_rm}.XLSX")
)
relacoes = le_arquivo_excel(
    os.path.join(os.getcwd(), "dados", f"GLINKSREL_TOTVS_RM_{versao_rm}.XLSX")
)


#### Organização de Relacionamentos

No contexto do TOTVS RM, a tabela `GLINKSREL` desempenha um papel crucial ao armazenar informações sobre relacionamentos entre tabelas, considerando tanto a relação de ida quanto a de volta. Para ilustrar, suponha que a tabela `SALUNO` se relacione com a tabela `PPESSOA` por meio das chaves `CODPESSOA` e `CODIGO`, respectivamente. Na tabela `GLINKSREL`, essas relações seriam representadas da seguinte maneira:

| `MASTERTABLE` | `CHILDTABLE` | `MASTERFIELD` | `CHILDFIELD` |
| :-----------: | :----------: | :-----------: | :----------: |
|   `PPESSOA`   |   `SALUNO`   |   `CODIGO`    | `CODPESSOA`  |
|   `SALUNO`    |  `PPESSOA`   |  `CODPESSOA`  |   `CODIGO`   |

A função `unifica_relacoes()` é que organiza essas tabelas de relacionamento em ordem alfabética e elimina duplicatas.

|    `A`    |   `B`    |       `LIGACOES`        |
| :-------: | :------: | :---------------------: |
| `PPESSOA` | `SALUNO` | (`CODIGO`, `CODPESSOA`) |


In [3]:
def unifica_relacoes():
    # Procura pelo arquivo pickle (ja processado pelo pandas)
    caminho_pickle = os.path.join(
        os.getcwd(), "dados", f"relacoes_unicas_{versao_rm}.pkl"
    )
    if os.path.isfile(caminho_pickle):
        return pd.read_pickle(caminho_pickle)

    # Ordena a linha e cria um dataframe
    relacoes_unicas = pd.DataFrame(np.sort(relacoes.iloc[:, :2]))
    # Remove duplicados
    relacoes_unicas = relacoes_unicas.drop_duplicates(ignore_index=True)

    # Cria nova coluna com um conjunto vazio
    relacoes_unicas["LIGACOES"] = [set() for _ in range(len(relacoes_unicas))]

    for [a, b, s] in relacoes_unicas.values:
        # Filtra as relacoes com infos de chaves extrangeiras
        A = relacoes.loc[
            (relacoes["MASTERTABLE"] == a) & (relacoes["CHILDTABLE"] == b),
            ["MASTERFIELD", "CHILDFIELD"],
        ]
        B = relacoes.loc[
            (relacoes["MASTERTABLE"] == b) & (relacoes["CHILDTABLE"] == a),
            ["CHILDFIELD", "MASTERFIELD"],
        ]

        # Salva as relacoes encontradas no conjunto
        for [a_chaves, b_chaves] in A.values:
            s.add((a_chaves, b_chaves))
        for [a_chaves, b_chaves] in B.values:
            s.add((a_chaves, b_chaves))

    # Salva num arquivo pickle para nao ter que recalcular
    relacoes_unicas.to_pickle(caminho_pickle)

    return relacoes_unicas


relacoes = unifica_relacoes()


In [4]:
print(f"{tabelas.iloc[:,0].drop_duplicates().shape[0]} tabelas definidas.")
print(f"{tabelas.shape[0]} colunas de tabelas definidas.")
print(f"{relacoes['LIGACOES'].map(len).sum()}", end="")
print(" relações definidas entre tabelas.")


8114 tabelas definidas.
128432 colunas de tabelas definidas.
16532 relações definidas entre tabelas.


In [5]:
tabelas_json = {}
for tabela, grupo in tabelas.groupby("TABELA"):
    tabelas_json[tabela] = {
        linha["COLUNA"]: linha["DESCRICAO"] for _, linha in grupo.iterrows()
    }

with open(f"./dados/tabelas_{versao_rm}.json", "w") as arquivo:
    json.dump(tabelas_json, arquivo, separators=(",", ":"))


In [ ]:
relacoes["LIGACOES"] = relacoes["LIGACOES"].apply(list)

relacoes_json = []
for _, linha in relacoes.iterrows():
    relacao = [linha[0], linha[1], linha["LIGACOES"]]
    relacoes_json.append(relacao)

with open(f"./dados/relacoes_{versao_rm}.json", "w") as arquivo:
    json.dump(relacoes_json, arquivo, separators=(",", ":"))
